In [3]:
'''
Plots showing the mean increase in mutation rates from mutator alleles 
that increase the mutation rate at modifier sites.
'''

'\nPlots showing the mean increase in mutation rates from mutator alleles \nthat increase the mutation rate at modifier sites.\n'

In [8]:
import pickle
import numpy as np
#inline matplotlib
import matplotlib as mpl
import seaborn as sns
from collections import defaultdict as ddict
from collections import namedtuple as nt
from collections import OrderedDict as odict
import os
import sys
sys.path.append("./..")
sys.path.append("/Users/will_milligan/PycharmProjects/Mutator_Project/Final Figs/self_inducing_results/new")

import mutator_classes as mc
import common_functions as cf
import stationary_distribution_aug as sd
plt = mpl.pyplot

In [9]:
# useful colors and functions
coral = [.973, .414, .575]
teal = [0,0.55,0.55]
charcoal = [0.3,0.3,0.3]

sns.set_style('whitegrid')
font_size = 10
mpl.rcParams['figure.dpi'] = 400
mpl.rcParams['axes.titlesize'] = 10
mpl.rcParams['axes.labelsize'] = 10
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['legend.fontsize'] = 10

In [11]:
# some helper stuff
u_bar = 1.25e-7
get_s = lambda p: 4*p.N*p.h*p.s*p.loci*p.phi
result_tuple = nt('result_tuple','mean mean_err var var_err within_var within_var_err M S phi parameters')

## get simulated results
# find files
indir = '/Users/will_milligan/PycharmProjects/Mutator_Project/Final Figs/self_inducing_results/new'
files = os.listdir(indir)

# append all results into a single dict
results = {}
for f in files:
    with open(os.path.join(indir,f),'rb') as fin:
        a = pickle.load(fin)
        results.update(a)
        
# make sure simulated results are ordered by phi values
ordered_keys = np.argsort([i.phi for i in results.keys()])

# calculate summaries
plotting_results = []
for index in ordered_keys:
    i = list(results.keys())[index]
    
    # get values to calculate mean and variance from
    freq_vals = []
    mean_err_vals = []
    var_err_vals = []
    
    # load sampled frequencies
    for v in results[i][4]:
        freq_vals.extend(v)
        mean_err_vals.append(np.mean(v))
        var_err_vals.append(np.var(v))
    
    # calcualte summaries
    mean_freq = np.mean(freq_vals)
    mean_err = np.std(mean_err_vals)/len(results[i][4])**(0.5)
    var_freq = np.var(freq_vals)
    var_err = np.std(var_err_vals)/len(results[i][4])**(0.5)
    
    plotting_results.append(result_tuple(S = get_s(i),
                                         mean = mean_freq,
                                         mean_err = mean_err,
                                         var = var_freq,
                                         var_err = var_err,
                                         within_var = np.nan,
                                         within_var_err = np.nan,
                                         M = i.M,
                                         parameters = i,
                                         phi = i.phi))
    

ModuleNotFoundError: No module named 'stationary_distribution'

In [ ]:
# get expected results

e_x = []
e_var1 = []
e_mean1 = []
phi_e = []

for S in np.logspace(-2,3,100):
    # change phi value
    i.phi = get_phi(param_dict,S)
    
    # calculate summary statistics
    statdist = sd.get_SD(i)
    emena = sum([k*p for k,p in statdist.items()])
    evara = sum([k**2*p for k,p in statdist.items()]) - emena**2
    phi_e.append(i.phi)
    
    # store results
    e_mean1.append(emena)
    e_var1.append(evara)
    e_x.append(S)

# make sure everything is sorted properly and calculate mutation rate moments not mutator freq moments
e_x_ord = np.argsort(e_x)
e_xvals = np.array(e_x)[e_x_ord]
phi_e = np.array(phi_e)

e_mean = ((np.array(e_mean1)[e_x_ord]))*np.array(phi_e)[e_x_ord]*2
e_var = (np.array(e_var1)[e_x_ord])*np.array(phi_e)[e_x_ord]**2*4
e_with = (np.array(e_with1)[e_x_ord])*np.array(phi_e)[e_x_ord]**2

In [ ]:
def common_modifications(ax,ylabel = '', title = ''):
    ax.set_xscale('log')
    ax.set_yscale('log')
    
    ax.spines['bottom'].set_color('k')
    ax.spines['top'].set_color('k')
    ax.spines['left'].set_color('k')
    ax.spines['right'].set_color('k')
    
    plt.sca(ax)
    plt.tick_params(bottom=True, top=False, left=True, right=False)
    plt.tick_params(bottom=True, top=False, left=True, right=False,which='minor')
    plt.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)
    plt.tick_params(labelbottom=False, labeltop=False, labelleft=False, labelright=False,which='minor')
    
    plt.xlabel('Scaled selection parameter ' + r'($4NLhs\phi$)',fontsize=10)
    plt.ylabel(ylabel)
    plt.title(title,pad=12,fontsize=10)
    plt.xlim([0.9e-2,1.1e3])
    plt.xticks(np.logspace(-2,3,6))

In [ ]:
# function to caluculate distance to fixed expected rate
def mean_distance(i):
    return np.abs((i.mean*i.phi*2+i.parameters.mutation_rate)/u_bar-1)
    
# find which simulated results had a mean rate  closest to fixed rate
best_M_values = {}
for i in plotting_results:
    distance = mean_distance(i)
    key_value = round(i.S,4)
    if key_value in best_M_values.keys() and distance < best_M_values[key_value][0]:
        best_M_values[key_value] = (distance,i)
    elif key_value not in best_M_values.keys():
        best_M_values[key_value] = (distance,i)
        

In [ ]:
# get expected value of M
def num_loci(mean_increase,mutation_rate = u_bar ,baseline=u_bar/2):
    return (mutation_rate-baseline)/mean_increase

# determine best value of M to use
bestM = [(i[1].M) for i in best_M_values.values()]

fig, ((ax1,trash,ax2),(ax3,trash2,trash3)) = plt.subplots(figsize=(6.5,9.75*6.5/8),nrows=2,ncols=3,gridspec_kw={'width_ratios': [20,5,20]})

# trash axis helps spacing other axes out
trash.axis('off')
trash2.axis('off')
trash3.axis('off')

markersize=6

# plot expected mutation rate relative to fixed rate
nvals = num_loci(e_mean)
ax2.plot(e_xvals,e_mean*nvals,color='k',ls='-',linewidth=1.5,alpha=0.5)
ax2.fill_between(x = [0.01,1000],y1=[1.15,1.15],y2=[0.85,0.85],color='grey',alpha=0.2)

# plot expected variance 
ax3.plot(e_xvals,e_var/4*nvals,color='k',ls='-',linewidth=1.5,alpha=0.5)

# plot simulated results from M values that have the smallest distance to the estimated rate
for distance,i in best_M_values.values():
    
    # mean rate
    ax2.errorbar(i.S,
         (i.mean*i.phi*2+i.parameters.mutation_rate)/u_bar,
         (i.mean_err*i.phi*2)/u_bar/(20)**(0.5)*2,
         ls='None',
         marker='.',
         markersize=markersize,
         color='k')
    
    # variance in mean rate
    ax3.errorbar(i.S,
         i.var*(i.phi*2)**2/(u_bar)**2,
         i.var_err*(i.phi*2)**2/(u_bar)**2/(20)**(0.5)*2,
         ls='None',
         marker='.',
         markersize=markersize,
         color='k')
    
# plot M*(\phi) assuming constant population size
ax1.plot(e_xvals,nvals,color='k',ls='-',linewidth=1.5,zorder=1,alpha=0.5)

# plot actual M used
ax1.scatter([i.S for d,i in best_M_values.values()],
                 [i.M for d,i in best_M_values.values()],
                 marker = '.',
                 color = 'k',
                 s = markersize**2,
                 zorder=10)

# do plotting stuff 
ax1.set_ylim([100,1e5])
common_modifications(ax1,
                     ylabel = r'Number of modifier sites $M\left(\phi\right)$',
                    title  = r'$\bf{A.}$ Num. of modifiers to condition on fixed expected rate')

common_modifications(ax2,
                     ylabel = 'Relative expected mutation rate ' + r'$E(u/\hat{u})$',
                     title  = r'$\bf{B.}$ Expected rate with given num. of modifiers')
ax2.set_ylim(0.7,1.3)
ax2.set_yscale('linear')

ax1.plot([],[],ls='-',color='k',label=r'Expectation with'+'\n'+'constant N')
ax1.errorbar([],[],[],ls='None',marker='.',markersize=5,color='k',label='Mean $\pm$ 2SE')
ax1.legend(framealpha=1,edgecolor='k',loc='upper left',handlelength = 1,borderaxespad=0.2,borderpad=0.2)
ax2.text(x = 0.011,y=1.11,s = r'$\pm 15\%$ of $\hat{u}$',color=[0.3,0.3,0.3])

common_modifications(ax3,
                     ylabel = 'Relative variance in mean rate ' + r'$V(\bar{u}/\hat{u})$',
                     title  = r'$\bf{C.}$ Variance conditional on fixed expected rate')

plt.subplots_adjust(wspace=0.,left = 0.05,right = 0.95,hspace=0.3)